In [ ]:
# Example notebook: football-data.org v4 API
#
# This notebook is meant to be a *tutorial / playground*.
# The concrete implementation (export) is run via:
# - code/main.py
#
# For data cleaning + modeling, use:
# - code/predictions.ipynb
#
# Install minimal dependency:
!python3 -m pip install -q requests


Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install pandas matplotlib

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 24.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 24.0 MB/s  0:00:00 eta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 9.9 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 30.4 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [matplotlib]1 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Shared setup for examples
import json
import os
import sys
from pathlib import Path

# Ensure we can import the implementation modules from ./code
_root = Path.cwd().resolve()
if _root.name == "code":
    _root = _root.parent
sys.path.insert(0, str(_root / "code"))

from football_data_api import (  # noqa: E402
    DEFAULT_TOKEN_ENV,
    FootballDataClient,
    RateLimitConfig,
    find_repo_root,
    load_dotenv_simple,
)


def show(title: str, payload: object) -> None:
    print(f"\n=== {title} ===")
    print(json.dumps(payload, indent=2, ensure_ascii=False)[:4000])


# Load .env (if present)
root = find_repo_root()
load_dotenv_simple(start_dir=root)

token = os.getenv(DEFAULT_TOKEN_ENV)
if not token:
    raise EnvironmentError(f"Missing {DEFAULT_TOKEN_ENV}. Put it in .env or export it.")

# Client: caching + rate limiting helps avoid 429.
client = FootballDataClient(
    token=token,
    cache_dir=root / "data" / "_api_cache",
    rate_limit=RateLimitConfig(min_seconds_between_calls=8.0),
)

print("Repo root:", root)
print("Cache dir:", root / "data" / "_api_cache")



In [5]:
# Example 1: fetch competition metadata (always allowed for many plans)
comp = client.get("/competitions/2014")
show("GET /competitions/2014", comp)




=== GET /areas (first 10) ===
{
  "count": 272,
  "filters": {},
  "areas": [
    {
      "id": 2000,
      "name": "Afghanistan",
      "countryCode": "AFG",
      "flag": null,
      "parentAreaId": 2014,
      "parentArea": "Asia"
    },
    {
      "id": 2001,
      "name": "Africa",
      "countryCode": "AFR",
      "flag": null,
      "parentAreaId": 2267,
      "parentArea": "World"
    },
    {
      "id": 2002,
      "name": "Albania",
      "countryCode": "ALB",
      "flag": null,
      "parentAreaId": 2077,
      "parentArea": "Europe"
    },
    {
      "id": 2004,
      "name": "Algeria",
      "countryCode": "ALG",
      "flag": null,
      "parentAreaId": 2001,
      "parentArea": "Africa"
    },
    {
      "id": 2005,
      "name": "American Samoa",
      "countryCode": "ASM",
      "flag": null,
      "parentAreaId": 2175,
      "parentArea": "Oceania"
    },
    {
      "id": 2006,
      "name": "Andorra",
      "countryCode": "AND",
      "flag": null,
      "pare

In [6]:
# Example 2: fetch matches for a specific season (may be restricted by plan)
# If you get 403 here, your subscription likely only allows newer seasons.
season = 2023
try:
    matches = client.get("/competitions/2014/matches", params={"season": season})
    show(f"GET /competitions/2014/matches?season={season}", matches)
except Exception as e:
    print("Request failed:", e)




=== GET /competitions (first 10) ===
{
  "count": 13,
  "filters": {
    "client": "Eamtc"
  },
  "competitions": [
    {
      "id": 2013,
      "area": {
        "id": 2032,
        "name": "Brazil",
        "code": "BRA",
        "flag": "https://crests.football-data.org/764.svg"
      },
      "name": "Campeonato Brasileiro Série A",
      "code": "BSA",
      "type": "LEAGUE",
      "emblem": "https://crests.football-data.org/bsa.png",
      "plan": "TIER_ONE",
      "currentSeason": {
        "id": 2474,
        "startDate": "2026-01-28",
        "endDate": "2026-12-02",
        "currentMatchday": 1,
        "winner": null
      },
      "numberOfAvailableSeasons": 10,
      "lastUpdated": "2024-09-13T16:55:53Z"
    },
    {
      "id": 2016,
      "area": {
        "id": 2072,
        "name": "England",
        "code": "ENG",
        "flag": "https://crests.football-data.org/770.svg"
      },
      "name": "Championship",
      "code": "ELC",
      "type": "LEAGUE",
      "embl

In [ ]:
# Example 3: run the real implementation (writes ./data/raw_data.csv)

!python3 main.py



Wrote 1140 match rows to /Users/mihaicoretchi/repos/AIA/data/raw_data.csv


In [58]:
!python3 analyze_raw_data.py

/Users/mihaicoretchi/repos/AIA/code/analyze_raw_data.py:49: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["utcDate"].dt.to_period("M").astype(str)
Loaded: /Users/mihaicoretchi/repos/AIA/data/raw_data.csv
Rows: 1,140
Date range: 2023-08-11 17:30:00+00:00 -> 2026-05-24 00:00:00+00:00
Wrote plots to: /Users/mihaicoretchi/repos/AIA/reports
